# **EECS 498 Deep Learning for Computer Vision (2019)**

These are my notes from Justin Johnson's course in 2019 which is heavily based on Stanford's CS 231N.

### **Lecture 12: Recurrent Neural Networks**

Recurrent neural networks are used to process sequences of inputs or outputs (video classification, machine translation, per-frame video classification). At every time step the RNN accepts an input that will update its hidden state $$h_t = f_W(h_{t-1}, x_t)$$ The same function and parameters are shared across each time step. 

#### **Vanilla RNNs**

A vanilla RNN has $h_t = \tanh(W_{hh} h_{t-1} + W_{xh}x_t)$ and $y_t = W_{hy} h_t$.

In practice Truncated Backpropogation Through Time is used where hidden states are carried through forever but backpropogation only traverses a portion of the sequence at a time.

When using RNNs for Image Captioning, the image is passed through a CNN and the output features $v$ are passed to an RNN. The update step now becomes $$h = \tanh(W_{xh} x + W_{hh} h + W_{ih}v) $$  The first input to the RNN is a $\verb|START|$ token.

RNNs suffer from exploding and vanishing gradients since in the backpropogation step. A solution is to use *gradient clipping* which will scale the gradient if its norm is too big.

#### **Long Short Term Memory (LSTM)**

LSTMs keep two hidden vectors at any time step and has update step $$\begin{align*} \begin{pmatrix} i \\ f \\ o \\ g\end{pmatrix} &= \begin{pmatrix} \sigma \\\sigma \\\sigma \\ \tanh \end{pmatrix} W \begin{pmatrix} h_{t-1} \\ x_t \end{pmatrix} \\
c_t &= f*c_{t-1} + i * g \\ h_{t} &= o * \tanh(c_t) \end{align*}$$ 


Here $g$ is the previous update step $\tanh(W_{xh} x + W_{hh}h)$, $i$ is the *input gate* that regulates what fraction of this to propogate, $f$ is the *forget gate* that regulates how much of the previous cell_state to forget, and $o$ is the *output gate* that regulates how much of the cell state should be shared with the output.

### **Lecture 13: Attention**

When working with sequence to sequence models we would like to not require our encoder network to encode a long sequence into a single vector. The attention mechanism allows the decoder to decide which parts of the input vector to look at to form the context vector at that time step.
 

We will compute alignment scores $e_{t,i} = f_{att}(s_{t-1}, h_i)$, where $s_{t}$ is the hidden state of the decoder at time step $t$ and $h_i$ is the hidden state of the encoder at time step $i$. The scores are passed through a softmax layer to obtain attention scores $a_{t,i}$ which satisfy $\sum_{i} a_{t,i} = 1$. The context vector at this step is $c_t = \sum_{i} a_{t,i} h_{i}$. This context vector is used in the decoder $s_t = g_U(y_{t-1}, h_{t-1}, c_t)$.

We can use the same attention mechanism to attend to other types of data.

##### **Image Captioning with RNNs and Attention**



Here we will pass the image through a CNN which will output a feature vector at each pixel $h_{(i,j)}$. Then we can use the attention mechanism to attend to these feature vectors $e_{t, (i,j)} = f_{att}(s_{t-1}, h_{(i,j)})$. Then we again pass through a softmax operation to get attention scores $a_{t, (i,j)}$ and obtain the context vector $c_{t} = \sum_{i,j} a_{t, (i,j)} h_{(i,j)}$. The new state is then computing by using this context vector and the input word at that step.

#### **Attention Layer**

We will abstract our attention mechanism into a more useful form so it can be more flexibly used.

One way to reframe our attention mechanism is that we have a query vector $q$, a collection of input vectors $X$, and a similarity function $f_{att}$. Then we compute similarity scores $e_i = f_{att}(q,X)$, use a softmax layer to get attention scores, and then output a weighted sum of the input.t

One modification is that we replace the neural network layer $f_{att}$ with a simple (scaled) dot product for efficiency. So we have $e_i = \frac{q \cdot X}{\sqrt{D_Q}}$ where $D_Q$ is the dimension of $q$. We also allow for multiple query vectors $Q$ at once. We similarly have $E = \frac{QX^T}{\sqrt{D_Q}}$. Then the softmax is applied column-wise for each query to obtain our attention matrix $A$, and our final output is $Y = AX$. 

We also want to separate the input vectors into keys and values by using learnable matrices $W_K, W_V$. This allows for the model to transform the input if it requires for easier computation and outputs a preferred form of the input. So now we have $K = XW_K$ giving the key vectors which are used to compute similarity and hence attention weights $A = \verb|softmax|(QK^T, dim=1)$ and this is used to obtain the output $Y = AV$ where $V = XW_V$ are the values.

In a *self attention layer* we have only one input matrix $X$ and we form queries within this input by introducing a learnable matrix $W_Q$ so that $Q = XW_Q$. Such a layer is permutation equivariant and hence the layer is independent of the order of input vectors. There are some situations where the order of the vectors matters, and in such situations a positional embedding is added to the input vectors.

A *masked self-attention-layer* doesn't let vectors look "ahead" in the sequence and only use information from the past. We do this by setting the corresponding $E_{ij}$ scores where $i < j$ to be $-\infty$ so that the softmax scores will transform them to $0$. Thus the network will not be allowed to attend to future parts of the sequence. This is useful in language modelling.

A *multi-head self attention layer* uses multiple "attention heads" independently in parallel. The input $X$ is broken into $H$ chunks and each chunk is input into a separate self attention layer in parallel. The outputs of these layers is concatenated. The hyperparameters are the dimension of the query matrix $D_Q$ and the number of heads $H$.

#### **Transformers**

The input to a transformer block is a set of vectors $x$ which is passed through a self attention layer (this will account for interactions between the inputs) and then a layer normalization after a residual connection is added. Then fully connected networks are applied independently on each vector and the outputs $y$ are obtained after a layer normalization with a residual connection. 

The only interaction between vectors occurs between the self attention layer, but because of how this layer is formed, it is still highly parallelizable.

A **Transformer** is a sequence of transformer blocks. In the original "Attention is all you need" paper, the encoder and gad have $6$ blocks with each self-attention layer having $6$ heads. 

Transformer models trained for language modelling can be fine tuned for other NLP tasks.

### **Lecture 15: Object Detection**

Here we input an image and the output is a set of detected objects along with a category label and a bounding box (center $=x,y$, width, height).

Some challenges in this task include:

* Need to output a variable number of outputs.
* Need to output only one box per object.
* Detection needs higher resolution than detection. Classification works at $224\times 224$ but detection often needs $800 \times 600$.

If we wanted to detect a single object we could pass the image through a CNN and then two parallel FC networks that predict class scores and box coordinates. The class scores use the softmax loss and the box coordinates use the $L_2$ loss. Then we take a weighted sum of these losses (the weight is trainable) to obtain a final loss.

But we require that the network output a different number of outputs per image depending on the number of detected objects.

A solution is to use a sliding window. Here we train a CNN to perform classification in sub-windows of the input image. However there are many possible bounding boxes and we also have to consider boxes at different scales so this is infeasible. For example, in a $800 \times 600$ image will involve $\sim 58$ million candidate boxes.

#### **Region Proposals and the R-CNN**

We can find a small set of boxes that are likely to cover all objects by training a separate network. The *R-CNN* (Region-based CNN) does this.

We input our image into a region proposal method that proposes candidate image regions at different sizes and aspect ratios. These region proposals are then warped to a fixed size like $224 \times 224$ and then passed through a classification CNN with $C + 1$ classes (the additional class is "no box"). Then the region proposal is "transformed" with trainable parameters to obtain the bounding box

A common transformation is as follows: given a region proposal $(p_x,p_y,p_h,p_y)$ we have $$\begin{align*} b_x &= p_x + p_w t_x & \text{    } b_y &= p_y + p_ht_y \\ b_w &= p_w\exp(t_w) & \text{    } b_h &= p_h \exp(t_h) \end{align*}$$ where $b_i$ are the output bounding box coordinates and $t_i$ are the trainable parameters.

So the pipeline at test time is as follows:

1. Run region proposal method to compute $\sim 2000$ region proposals.
2. Resize each region to $224 \times 224$ and run independently through $CNN$ to predict class scores and b-box transformation.
3. Use scores to select a subset of region proposals to output.
4. Compare with ground truth boxes.

We compare our prediction with the ground truth box by using a simple intersection-over-union (IoU) or *Jaccard similarity* score where we compute the ratio of the area of intersection of the two boxes with the ratio of the area of the union. A score above $0.5$ is "decent", a score above $0.7$ is "pretty good" and a score above $0.9$ is "almost perfect".

We also perform non-maximal suppression to avoid overlapping many overlapping detections (possible of the same object). In this algorithm at each stage we pick the next highest scoring box and eliminate any lower scoring box with $IoU$ score bigger than a given threshold (like 0.7). However this becomes an issue in situations like counting the number of people in a crowd where there are many people close to each other.

To evaluate object detectors we use *Mean Average Precision (mAP)*. Here we run the object detector on all test images (with NMS). For each category we compute average precision (area under the precision-recall curve). More precisely, 

1. We match each detection with some ground truth box and if it has IoU $> 0.5$ we mark it as positive and eliminate the ground truth box. Otherwise we mark it as negative. Then  we plot a point on the precision recall curve. 
2. We calculate the average precision (AP) by finding the area under PR curve.

We then repeat this for each category and take the average of these values to get the MAP. In practice we compute this for different IoU thresholds and take the average of these to get a COCO MAP.



R-CNNs are slow since they need $\sim 2000$ forward passes for each image and so the above method is actually called "Slow R-CNN".

#### **Fast R-CNN**

Here we swap the convolution and region warping. We pass the image through a backbone CNN network to get image features. We then perform a region proposal method onto the feature map, crop and resize features, and then pass it to a lightweight per-region network to obtain bounding boxes and classes.

We need to crop and resize images in a way that is differentiable. Two methods for this are $\verb|ROI Pool|$ and $\verb|ROI align|$.

Fast R-CNN recommends using a smooth $L_1$ loss $f(x) = \begin{cases} 0.5x^2 &\text{ if } \lvert x \rvert < 1\\ \lvert x\rvert - 0.5 & \text{ otherwise}\end{cases}$

Faster R-CNN replaces selective search with a Region Proposal network by using anchor boxes of different scales to identify multiple objects present in the same window.